# 1. Load modules

In [131]:
import arcpy
from arcpy.sa import *

# 2. Define airport locations and runway angles

In [120]:
airport_name_list = ['Heathrow', \
                     'Gatwick', \
                     'Manchester', \
                     'Stansted', \
                     'Edinburgh', \
                     'Birmingham', \
                     'Bristol', \
                     'Glasgow', \
                     'Aberdeen', \
                     'EastMidlands', \
                     'LondonCity', \
                     'BelfastInt', \
                     'Newcastle', \
                     'LeedsBradford', \
                     'Liverpool',\
                     'Cardiff']

#airport_name_list = ['Cardiff']

dict = {'Heathrow':{'lat': 51.470022, 'lon':-0.454296, 'angle': 0},\
        'Gatwick':{'lat': 51.153662, 'lon':-0.182063, 'angle': 8},\
        'Manchester':{'lat': 53.358803, 'lon':-2.27273, 'angle': 25},\
        'Stansted':{'lat': 51.886018, 'lon':0.238866, 'angle': 34},\
        'Edinburgh':{'lat': 55.950785, 'lon':-3.361453, 'angle': 19},\
        'Birmingham':{'lat': 52.452382, 'lon':-1.743507, 'angle': 137},\
        'Bristol':{'lat': 51.38363, 'lon':-2.713546, 'angle': 3},\
        'Glasgow':{'lat': 55.869074, 'lon':-4.435053, 'angle': 28},\
        'Aberdeen':{'lat': 57.203698, 'lon':-2.200162, 'angle': 127},\
        'EastMidlands':{'lat': 52.829374, 'lon':-1.332134, 'angle': 0},\
        'LondonCity':{'lat': 51.504844, 'lon':0.049518, 'angle': -2},\
        'BelfastInt':{'lat': 54.654173, 'lon':-6.224699, 'angle': 15},\
        'Newcastle':{'lat': 55.038524, 'lon':-1.693079, 'angle': 14},\
        'LeedsBradford':{'lat': 53.867943, 'lon':-1.661531, 'angle': -34},\
        'Liverpool':{'lat': 53.336707, 'lon':-2.85744, 'angle': 3},\
        'Cardiff':{'lat': 51.396667, 'lon':-3.343333, 'angle': -17}}

# 3. Main code for creating polygons

In [125]:
for airport_name in airport_name_list:
    airport = dict[airport_name]
    lat = airport.get('lat')
    lon = airport.get('lon')
    angle = airport.get('angle')

    distance = 25000

    gcs = arcpy.SpatialReference(4326)
    centre = arcpy.PointGeometry(arcpy.Point(lat,lon),gcs)

    point2 = centre.pointFromAngleAndDistance(angle, distance, 'GEODESIC')
    print('point2 = {:4f},{:.4f}'.format(
            point2.firstPoint.X,
            point2.firstPoint.Y))

    point1 = centre.pointFromAngleAndDistance(angle+180, distance, 'GEODESIC')
    print('point1 = {:4f},{:.4f}'.format(
            point1.firstPoint.X,
            point1.firstPoint.Y))

    coordinates = [(point1.firstPoint.Y, point1.firstPoint.X), (point2.firstPoint.Y, point2.firstPoint.X)]
    #coordinates = [(centre.firstPoint.Y, centre.firstPoint.X), (point2.firstPoint.Y, point2.firstPoint.X)]


    arcpy.CheckOutExtension('spatial')
    result = arcpy.management.CreateFeatureclass('C:/Users/gemma.daron/OneDrive - Met Office/ArcGISProProjects/ModeS', 
                                            '{0}_line.shp'.format(airport_name),
                                            'POLYLINE',
                                            '',
                                            '',
                                            '',
                                            '4326') # 4326 WGS1984 # 4277 OSGB36
    outPolyExtent = result[0]
    # Use Insert cursor to add new geometry to feature class Write feature to new feature class
    with arcpy.da.InsertCursor(outPolyExtent, ['SHAPE@']) as cursor:
        cursor.insertRow([coordinates])
    
    arcpy.CheckInExtension('Spatial')

    arcpy.Buffer_analysis('{0}_line'.format(airport_name), '{0}_buffer'.format(airport_name), '3000 METERS')

point2 = 51.330909,-3.1271
point1 = 51.462453,-3.5595
